In [102]:
module EV3

using ZMQ

export Motor, run_forever, stop

SYS_ROOT = "/sys"

abstract LegoDevice
abstract LegoInput <: LegoDevice
abstract LegoOutput <: LegoDevice

abstract AbstractNode

immutable LocalNode <: AbstractNode
    path::AbstractString
end

immutable RemoteNode <: AbstractNode
    path::AbstractString
    hostname::AbstractString
    socket::Socket
end

immutable Motor <: LegoOutput 
    port::AbstractString
    node::AbstractNode
end

immutable Sensor <: LegoInput
    port::AbstractString
    node::AbstractNode
end

function Motor(port::AbstractString, hostname::Union{Void, AbstractString}=nothing)
    if hostname == nothing
        Motor(port, LocalNode("$(SYS_ROOT)/class/tacho-motor/$(port)"))
    else
        ctx = Context()
        socket = Socket(ctx, REQ)
        ZMQ.connect(socket, "tcp://$(hostname):5555")
        Motor(port, RemoteNode("$(SYS_ROOT)/class/tacho-motor/$(port)", 
        hostname,
        socket))
    end
end

function Sensor(port::AbstractString, hostname::Union{Void, AbstractString}=nothing)
    if hostname == nothing
        Sensor(port, LocalNode("$(SYS_ROOT)/class/lego-sensor/$(port)"))
    else
        ctx = Context()
        socket = Socket(ctx, REQ)
        ZMQ.connect(socket, "tcp://$(hostname):5555")
        Sensor(port, RemoteNode("$(SYS_ROOT)/class/lego-sensor/$(port)", 
        hostname,
        socket))
    end
end

function run_forever(motor::Motor, speed=100)
    write("on", motor.node, "speed_regulation")
    write(string(speed), motor.node, "speed_sp")
    write("run-forever", motor.node, "command")
end

function stop(motor::Motor)
    write("stop", motor.node, "command")
end

function write(data::AbstractString, node::LocalNode, file::AbstractString)
    open(f -> write(f, data), joinpath(node.path, file))
end

function write(data::AbstractString, node::RemoteNode, file::AbstractString)
    command = "w:$(data):$(node.path)/$(file)"
    ZMQ.send(node.socket, command)
    msg = ZMQ.recv(node.socket)
    out = convert(IOStream, msg)
    seek(out, 0)
    return bytestring(out)
end

function read(node::RemoteNode, file::AbstractString)
    command = "r:$(node.path)/$(file)"
    ZMQ.send(node.socket, command)
    msg = ZMQ.recv(node.socket)
    out = convert(IOStream, msg)
    seek(out, 0)
    return bytestring(out)
end

function get_value0(sensor::Sensor)
    read(sensor.node, "value0")
end

function get_position(motor::Motor)
    read(motor.node, "position")
end

end

EV3

In [103]:
import EV3

In [106]:
motor = EV3.Motor("motor1", "192.168.1.27")

EV3.Motor("motor1",EV3.RemoteNode("/sys/class/tacho-motor/motor1","192.168.1.27",ZMQ.Socket(Ptr{Void} @0x00007fdfc5b59800,Base._FDWatcher(Ptr{Void} @0x00007fdfc8ce1c20,(1,0),Condition(Any[]),(false,false)))))

In [112]:
EV3.get_position(motor)

"702\n"

In [109]:
EV3.run_forever(motor, 300)

""

In [110]:
EV3.stop(motor)

""

In [91]:
us = EV3.Sensor("sensor0", "192.168.1.27")

EV3.Sensor("sensor0",EV3.RemoteNode("/sys/class/lego-sensor/sensor0","192.168.1.27",ZMQ.Socket(Ptr{Void} @0x00007fdfc51aa000,Base._FDWatcher(Ptr{Void} @0x00007fdfc90831a0,(1,0),Condition(Any[]),(false,false)))))

In [92]:
while true
    println(parse(Int, EV3.get_value0(us)))
end

93
93
93
93
93
93
93
93
93
93
93
93
93
86
107
67
77
334
326
326
2550
2550
2550
2550
2550
2550
2550
2550
2550
546
546
546
2550
2550
2550
2550
2550
2550
2550
2550
2550
1564
1533
1528
1520
1513
1507
1498
1498
1495
1495
1498
1527
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
1894
1900
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
1534
1528
1543
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
130
326
326
199
189
166
164
139
139
143
149
147
156
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
2550
326
326
326
326
183
183
183
183
183
183
183
1517
1534
2550
2550
2550
2550


LoadError: LoadError: InterruptException:
while loading In[92], in expression starting on line 1

In [1]:
using ZMQ

In [2]:
ctx = Context(1)
sock = Socket(ctx, REQ)
ZMQ.connect(sock, "tcp://192.168.1.25:5555")

function send_command(sock::Socket, command::AbstractString)
    ZMQ.send(sock, command)
    msg = ZMQ.recv(sock)
    out = convert(IOStream, msg)
    seek(out, 0)
    return bytestring(out)
end


send_command (generic function with 1 method)

 in depwarn at deprecated.jl:73
 in call at deprecated.jl:50
 in include_string at loading.jl:266
 in execute_request_0x535c5df2 at /Users/rdeits/.julia/IJulia/src/execute_request.jl:177
 in eventloop at /Users/rdeits/.julia/IJulia/src/IJulia.jl:141
 in anonymous at task.jl:447
while loading In[2], in expression starting on line 1


In [7]:
send_command(sock, "echo 50 > /sys/class/tacho-motor/motor0/duty_cycle_sp")
send_command(sock, "echo run-forever > /sys/class/tacho-motor/motor0/command")


""

In [8]:
send_command(sock, "echo stop > /sys/class/tacho-motor/motor0/command")

""

In [5]:
send_command(sock, "echo on > /sys/class/tacho-motor/motor0/speed_regulation")
for i = 1:10
    print(i)
    send_command(sock, "echo $(i*20) > /sys/class/tacho-motor/motor0/speed_sp")
    send_command(sock, "echo run-forever > /sys/class/tacho-motor/motor0/command")
    sleep(1)
end
send_command(sock, "echo stop > /sys/class/tacho-motor/motor0/command")

12345678910

""

In [9]:
send_command(sock, "cat /sys/class/tacho-motor/motor0/position")

"4986\n"

In [ ]:
import zmq
import subprocess
import os.path

context = zmq.Context()
socket = context.socket(zmq.REP)
socket.bind("tcp://*:5555")

while True:
    message = socket.recv()
    print("Received request: %s" % message)
    if message.startswith("w:"):
        prefix, data, path = message.split(":")
        path = os.path.abspath(path)
        command = "echo %s > %s" % (data, path)
    elif message.startswith("r:")
        prefix, path = message.split(":")
        path = os.path.abspath(path)
        command = "cat %s" % path
    else:
        print "unrecognized prefix: %s" % message[:2]
        continue
    print("Running: %s" % command)
    try:
        response = subprocess.check_output(command, shell=True)
        socket.send(response)
    except subprocess.CalledProcessError as e:
        print e